## 여러 CSV 파일 전부 합치기

- 이 파일이 위치하는 폴더(작업 폴더)안에 폴더를 하나 만들고 그 안에 CSV 파일 전부 넣으면 됩니다.

In [13]:
import os
import pandas as pd
import re
import datetime

now = datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')

# ★★★★★★★★★★★★★★★★★★★★★★★★
# < 직접 수정하는 부분 >
# ★★★★★★★★★★★★★★★★★★★★★★★★
input_folder = "full_file"       # CSV 파일들이 있는 폴더

output_file = f"merged_data_{now}.csv"  # 저장할 파일명

In [14]:
# 특수문자 제거 함수
def remove_special_characters(text):
    return re.sub(r'[^\w\s]', '', text) if isinstance(text, str) else text

# 한글과 영어 순서를 정렬하는 함수 - 한글 제목과 영어 제목이 서로 다른 순서로 정렬되어있는 경우에도 중복을 제거할 수 있게끔 설정정
def normalize_title(title):
    if isinstance(title, str):
        # 한글과 영어를 분리
        korean = " ".join(re.findall(r'[가-힣]+', title))
        english = " ".join(re.findall(r'[a-zA-Z]+', title))
        # 한글과 영어를 알파벳 순으로 정렬
        return " ".join(sorted([korean, english])).strip()
    return title

In [15]:
all_data = []

# input_folder에 있는 모든 CSV 파일 읽기
for file in os.listdir(input_folder):
    if file.endswith('.csv'):
        file_path = os.path.join(input_folder, file)
        df = pd.read_csv(file_path)

        # 열 이름에 따라 처리
        if 'abstract' in df.columns:  # 새로운 형식의 CSV 파일
            df.rename(columns={'abstract': '요약', '날짜': '발행연도'}, inplace=True)
            df['발행연도'] = df['발행연도'].astype(str).str[:4]  # 날짜에서 연도만 추출

        # 제목 열과 요약 열에서 특수문자 제거
        if '제목' in df.columns:
            df['제목'] = df['제목'].apply(remove_special_characters)
            df['제목'] = df['제목'].apply(normalize_title)  # 제목 정규화
        if '요약' in df.columns:
            df['요약'] = df['요약'].apply(remove_special_characters)

        # 필요한 열만 유지, 누락된 열 추가
        for col in ['제목', '키워드', '발행연도', '요약']:
            if col not in df.columns:
                df[col] = None
        df = df[['제목', '키워드', '발행연도', '요약']]

        all_data.append(df)

# 모든 데이터를 하나로 합치기
merged_data = pd.concat(all_data, ignore_index=True)

# 제목 기준으로 중복 제거
if '제목' in merged_data.columns:
    merged_data = merged_data.drop_duplicates(subset=['제목'])

# 결과를 새로운 CSV 파일로 저장
merged_data.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"Merged and deduplicated data saved to {output_file}")

Merged and deduplicated data saved to merged_data_2025-01-16_214512.csv
